In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
#from textblob import TextBlob

In [2]:
spark = SparkSession.builder.appName("CSVtoTable").getOrCreate()

df = (spark.read.format("csv")
.option('mode','DROPMALFORMED')
.options(header = True, inferSchema = True, sep=",",multiLine=True)
.load("/Users/sarrabenyahia/Documents/GitHub/Twitter-WebScraping/webscraped_VALERO_ENERGY_CORP.csv")
.cache()) # Keep the dataframe in memory for faster processing 

23/01/25 17:00:04 WARN Utils: Your hostname, MacBook-Air-de-Sarra.local resolves to a loopback address: 127.0.0.1; using 172.20.10.7 instead (on interface en0)
23/01/25 17:00:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/25 17:00:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Cleaning des données

#Dates
df = df.na.fill(0, subset=['ReplyCount','RetweetCount','LikeCount'])
#Fillna de reply, retweet et like en à

df = df.select(*[col(c).cast("integer").alias(c) if c in ['ReplyCount','RetweetCount','LikeCount'] else c for c in df.columns])
df = df.withColumn("PostDate", to_date(df["PostDate"]))

#Cleaning du texte
df = df.withColumn("TweetText", trim(regexp_replace("TweetText", "[\n]+", " ")))
df = df.withColumn("TweetText", trim(regexp_replace("TweetText", "https?://[^ ]+", "")))
df = df.withColumn("TweetText", trim(regexp_replace("TweetText", "@[^ ]+", "")))
df = df.withColumn("TweetText", trim(regexp_replace("TweetText", "#[^ ]+", "")))
df = df.withColumn("TweetText", trim(regexp_replace("TweetText", "[^A-Za-z0-9 ]", "")))
df = df.withColumn("TweetText", regexp_extract("TweetText", "(2021|2017|2018|2019|2020|2022).*", 0))
df = df.withColumn("TweetText", regexp_replace("TweetText", "[^a-zA-Z\\s]", ""))
df = df.na.drop(subset="TweetText") 

#On enlève le @ du handle
df = df.withColumn("Handle", trim(regexp_replace("Handle", "@", "")))

In [4]:
""" 
A ajouter en spark : 

# Repeating words like hurrrryyyyyy
rpt_regex = re.compile(r"(.)\1{1,}", re.IGNORECASE);
def rpt_repl(match):
	return match.group(1)+match.group(1)

def remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words
	for remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", "\"", "=", "&", ";", "%", "$",
																	"@", "%", "^", "*", "(", ")", "{", "}",
																	"[", "]", "|", "/", "\\", ">", "<", "-",
																	"!", "?", ".", "'",
																	"--", "---", "#"]):
		tweets.loc[:, "text"].replace(remove, "", inplace=True)
	return tweets


"""
    

' \nA ajouter en spark : \n\n# Repeating words like hurrrryyyyyy\nrpt_regex = re.compile(r"(.)\x01{1,}", re.IGNORECASE);\ndef rpt_repl(match):\n\treturn match.group(1)+match.group(1)\n\n\ndef remove_na(self, tweets):\n\treturn tweets[tweets["text"] != "Not Available"]\n\ndef remove_special_chars(self, tweets):  # it unrolls the hashtags to normal words\n\tfor remove in map(lambda r: regex.compile(regex.escape(r)), [",", ":", """, "=", "&", ";", "%", "$",\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"@", "%", "^", "*", "(", ")", "{", "}",\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"[", "]", "|", "/", "\\", ">", "<", "-",\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"!", "?", ".", "\'",\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t"--", "---", "#"]):\n\t\ttweets.loc[:, "text"].replace(remove, "", inplace=True)\n\treturn tweets\n\n\n'

In [5]:
df.printSchema()


root
 |-- User: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- PostDate: date (nullable = true)
 |-- TweetText: string (nullable = true)
 |-- ReplyCount: integer (nullable = true)
 |-- RetweetCount: integer (nullable = true)
 |-- LikeCount: integer (nullable = true)



In [6]:
df.dtypes


[('User', 'string'),
 ('Handle', 'string'),
 ('PostDate', 'date'),
 ('TweetText', 'string'),
 ('ReplyCount', 'int'),
 ('RetweetCount', 'int'),
 ('LikeCount', 'int')]

In [7]:
df.count()

1813

In [8]:
df.show()

+--------------------+---------------+----------+--------------------+----------+------------+---------+
|                User|         Handle|  PostDate|           TweetText|ReplyCount|RetweetCount|LikeCount|
+--------------------+---------------+----------+--------------------+----------+------------+---------+
|        Utah Hamrick|       uhamrick|2021-09-25|Playing an outdoo...|      null|        null|        1|
|        BIC Magazine|    BICMagazine|2021-09-20|With  years of ex...|      null|        null|        1|
|AJ Blackston - Fi...|    AJBlackston|2021-09-17|VALERO ENERGY COR...|      null|        null|     null|
|      stocksoftheday|stocksoftheday1|2021-09-13|VLO Valero Energy...|      null|        null|     null|
|     Market Engineer|_MarketEngineer|2021-09-11|VLO Valero Energy...|      null|        null|     null|
|AJ Blackston - Fi...|    AJBlackston|2021-09-09|VALERO ENERGY COR...|      null|        null|     null|
|           Spot Etal|       EtalSpot|2021-09-08| While

In [9]:
df.groupBy("ReplyCount").count().show()

+----------+-----+
|ReplyCount|count|
+----------+-----+
|      null| 1773|
|         1|   35|
|         3|    1|
|         4|    1|
|         2|    3|
+----------+-----+



In [10]:
df.groupBy("RetweetCount").count().show()

+------------+-----+
|RetweetCount|count|
+------------+-----+
|          31|    1|
|          12|    1|
|        null| 1710|
|           1|   62|
|           6|    2|
|           3|    5|
|           5|    2|
|          15|    1|
|          17|    2|
|           4|    4|
|           7|    2|
|          11|    1|
|          14|    1|
|           2|   19|
+------------+-----+



In [11]:
df.groupBy("LikeCount").count().show()
#ponderer le poids du sentiment par rapport au nombre de like

+---------+-----+
|LikeCount|count|
+---------+-----+
|       12|    2|
|     null| 1692|
|        1|   58|
|       16|    1|
|        6|    2|
|        3|   13|
|        5|    5|
|       19|    1|
|       17|    1|
|        4|   12|
|        7|    2|
|       10|    2|
|       32|    1|
|       14|    1|
|        2|   20|
+---------+-----+

